# Example: Optimal adversaries for dense MNIST model


## Building and training the neural network

In [1]:
#Import requisite packages
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

Show how to load the dataset for training

In [3]:
#Set training and test batch sizes
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}

#Build DataLoaders for training and test sets
dataset1 = datasets.MNIST('../data', train=True, transform=transforms.ToTensor(),download=True)
dataset2 = datasets.MNIST('../data', train=False, transform=transforms.ToTensor(),download=True)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

100.0%


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



102.8%


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100.0%


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



112.7%

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



Define model

In [4]:
hidden_size = 50

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1  = nn.Linear(784, hidden_size)
        self.hidden2  = nn.Linear(hidden_size, hidden_size)
        self.output  = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)      
        return x

Define train and test functions

In [5]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28*28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader):
    model.eval()
    test_loss = 0; correct = 0
    criterion = nn.NLLLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 28*28))
            test_loss += criterion(output, target).item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))            

Train model on dataset

In [6]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.307344
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.272494
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.381385
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.210876
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.137464

Test set: Average loss: 0.1602, Accuracy: 9499/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.075432
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.141608
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.074647
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.070915
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.086606

Test set: Average loss: 0.1215, Accuracy: 9630/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.116157
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.041469
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.041615
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.036402
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.097630

Test set: Average loss: 0.1103, Accuracy: 9678/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.127977
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Building the MIP formulation

Need to export to ONNX, the PyTorch ONNX exporter needs to write to a file so we generate a temporary file.

In [7]:
import torch.onnx
import tempfile
from omlt.io.onnx import write_onnx_model_with_bounds, load_onnx_neural_network_with_bounds

We also define bounds on variables

In [8]:
problem_index = 0
image = dataset2[problem_index][0].view(-1,28*28).detach().numpy()
label = dataset2[problem_index][1]

In [9]:
epsilon_infty = 1e-2
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)
input_bounds = [(float(l), float(u)) for l, u in zip(lb[0], ub[0])]

PyTorch needs to trace the model execution to export it, so we defined a dummy input tensor.

In [10]:
x = dataset2[problem_index][0].view(-1,28*28)

Now we can write the ONNX model and load it back.

In [11]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as f:
    torch.onnx.export(
        model,
        x,
        f,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load back
    network_definition = load_onnx_neural_network_with_bounds(f.name)

Create Pyomo model

In [19]:
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import FullSpaceNNFormulation
from omlt.neuralnet.activations import linear_activation_constraint

OMLT doesn't include a formulation for sigmoid, so define it here

In [20]:
formulation = FullSpaceNNFormulation(network_definition,
    activation_constraints={'logsoftmax': linear_activation_constraint})

m = pyo.ConcreteModel()

m.nn = OmltBlock()
m.nn.build_formulation(formulation)

In [21]:
m.pprint()

1 Block Declarations
    nn : Size=1, Index=None, Active=True
        5 Set Declarations
            input_assignment_index : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :  784 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168

                271 :    0.9860784411430359 :     0 :                  1.0 : False : False :  Reals
                272 :    0.9860784411430359 :     0 :                  1.0 : False : False :  Reals
                273 :    0.5390196442604065 :     0 :   0.5590196251869202 : False : False :  Reals
                274 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                275 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                276 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                277 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                278 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                279 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals
                280 :                   0.0 :     0 : 0.009999999776482582 : False : False :  Reals


                Key : Lower : Body                                   : Upper : Active
                  0 :   0.0 :     nn.scaled_inputs[0] - nn.inputs[0] :   0.0 :   True
                  1 :   0.0 :     nn.scaled_inputs[1] - nn.inputs[1] :   0.0 :   True
                  2 :   0.0 :     nn.scaled_inputs[2] - nn.inputs[2] :   0.0 :   True
                  3 :   0.0 :     nn.scaled_inputs[3] - nn.inputs[3] :   0.0 :   True
                  4 :   0.0 :     nn.scaled_inputs[4] - nn.inputs[4] :   0.0 :   True
                  5 :   0.0 :     nn.scaled_inputs[5] - nn.inputs[5] :   0.0 :   True
                  6 :   0.0 :     nn.scaled_inputs[6] - nn.inputs[6] :   0.0 :   True
                  7 :   0.0 :     nn.scaled_inputs[7] - nn.inputs[7] :   0.0 :   True
                  8 :   0.0 :     nn.scaled_inputs[8] - nn.inputs[8] :   0.0 :   True
                  9 :   0.0 :     nn.scaled_inputs[9] - nn.inputs[9] :   0.0 :   True
                 10 :   0.0 :   nn.scaled_inputs[10] -

                Key : Lower : Body                                                     : Upper : Active
                  0 :   0.0 :     nn.scaled_inputs[0] - nn.layer[140391272872016].z[0] :   0.0 :   True
                  1 :   0.0 :     nn.scaled_inputs[1] - nn.layer[140391272872016].z[1] :   0.0 :   True
                  2 :   0.0 :     nn.scaled_inputs[2] - nn.layer[140391272872016].z[2] :   0.0 :   True
                  3 :   0.0 :     nn.scaled_inputs[3] - nn.layer[140391272872016].z[3] :   0.0 :   True
                  4 :   0.0 :     nn.scaled_inputs[4] - nn.layer[140391272872016].z[4] :   0.0 :   True
                  5 :   0.0 :     nn.scaled_inputs[5] - nn.layer[140391272872016].z[5] :   0.0 :   True
                  6 :   0.0 :     nn.scaled_inputs[6] - nn.layer[140391272872016].z[6] :   0.0 :   True
                  7 :   0.0 :     nn.scaled_inputs[7] - nn.layer[140391272872016].z[7] :   0.0 :   True
                  8 :   0.0 :     nn.scaled_inputs[8] - nn.layer

                            Key : Lower : Body                                                                                                                                                     : Upper : Active
                              0 :  -Inf :     nn.layer[140391272458640].z[0] - (nn.layer[140391272458640].zhat[0] - nn.layer[140391272458640]._big_m_lb[0]*(1.0 - nn.layer[140391272458640].q[0])) :   0.0 :   True
                              1 :  -Inf :     nn.layer[140391272458640].z[1] - (nn.layer[140391272458640].zhat[1] - nn.layer[140391272458640]._big_m_lb[1]*(1.0 - nn.layer[140391272458640].q[1])) :   0.0 :   True
                              2 :  -Inf :     nn.layer[140391272458640].z[2] - (nn.layer[140391272458640].zhat[2] - nn.layer[140391272458640]._big_m_lb[2]*(1.0 - nn.layer[140391272458640].q[2])) :   0.0 :   True
                              3 :  -Inf :     nn.layer[140391272458640].z[3] - (nn.layer[140391272458640].zhat[3] - nn.layer[140391272458640]._big_m_lb[

                            Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

                            Key : Lower : Body                                                                                                                                                     : Upper : Active
                              0 :  -Inf :     nn.layer[140391416300688].z[0] - (nn.layer[140391416300688].zhat[0] - nn.layer[140391416300688]._big_m_lb[0]*(1.0 - nn.layer[140391416300688].q[0])) :   0.0 :   True
                              1 :  -Inf :     nn.layer[140391416300688].z[1] - (nn.layer[140391416300688].zhat[1] - nn.layer[140391416300688]._big_m_lb[1]*(1.0 - nn.layer[140391416300688].q[1])) :   0.0 :   True
                              2 :  -Inf :     nn.layer[140391416300688].z[2] - (nn.layer[140391416300688].zhat[2] - nn.layer[140391416300688]._big_m_lb[2]*(1.0 - nn.layer[140391416300688].q[2])) :   0.0 :   True
                              3 :  -Inf :     nn.layer[140391416300688].z[3] - (nn.layer[140391416300688].zhat[3] - nn.layer[140391416300688]._big_m_lb[

                            Key : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          